In [ ]:
#|default_exp data.transforms

# Data Transforms> Data augmentation and transformation functions for time series

This module provides data augmentation techniques:- Gaussian noise injection- Magnitude scaling/warping- Time warping- Window slicing- Mixup

In [ ]:
import tsai_rsimport numpy as np

## Gaussian NoiseAdd random Gaussian noise to time series for augmentation.

In [ ]:
# Load sample dataX_train, y_train, X_test, y_test = tsai_rs.get_UCR_data('ECG200')X_train_std = tsai_rs.ts_standardize(X_train)

In [ ]:
# Add Gaussian noise using tsai_rsX_noisy = tsai_rs.add_gaussian_noise(X_train_std, scale=0.1)print(f"Original std: {np.std(X_train_std):.4f}")print(f"Noisy std: {np.std(X_noisy):.4f}")print(f"Noise difference mean: {np.mean(np.abs(X_noisy - X_train_std)):.4f}")

## Magnitude ScalingScale the magnitude of time series by a random factor.

In [ ]:
# Magnitude scaling using tsai_rsX_scaled = tsai_rs.mag_scale(X_train_std, scale=0.2)print(f"Original range: [{X_train_std.min():.4f}, {X_train_std.max():.4f}]")print(f"Scaled range: [{X_scaled.min():.4f}, {X_scaled.max():.4f}]")

## Additional Transform Functions

In [ ]:
def time_warp(X, sigma=0.2, knot=4):    """Apply time warping augmentation."""    X = np.asarray(X)    n_samples, n_channels, seq_len = X.shape        # Create warping path    orig_steps = np.arange(seq_len)        warped = np.zeros_like(X)    for i in range(n_samples):        # Random warping for each sample        warp_steps = np.cumsum(np.random.normal(loc=1, scale=sigma, size=seq_len))        warp_steps = warp_steps / warp_steps[-1] * (seq_len - 1)                for j in range(n_channels):            warped[i, j] = np.interp(orig_steps, warp_steps, X[i, j])        return warpeddef window_slice(X, reduce_ratio=0.9):    """Slice a window from the time series."""    X = np.asarray(X)    n_samples, n_channels, seq_len = X.shape    target_len = int(seq_len * reduce_ratio)        sliced = np.zeros((n_samples, n_channels, target_len))    for i in range(n_samples):        start = np.random.randint(0, seq_len - target_len + 1)        sliced[i] = X[i, :, start:start + target_len]        return sliceddef mixup(X, y, alpha=0.2):    """Apply mixup augmentation."""    X = np.asarray(X)    y = np.asarray(y)    n = len(X)        # Random permutation for mixing    indices = np.random.permutation(n)        # Sample lambda from Beta distribution    lam = np.random.beta(alpha, alpha, n)    lam = lam.reshape(-1, 1, 1)        X_mixed = lam * X + (1 - lam) * X[indices]    y_mixed = lam.squeeze() * y + (1 - lam.squeeze()) * y[indices]        return X_mixed, y_mixed

In [ ]:
# Test transformsX_warped = time_warp(X_train_std[:10], sigma=0.1)print(f"Time warped shape: {X_warped.shape}")X_sliced = window_slice(X_train_std[:10], reduce_ratio=0.8)print(f"Window sliced shape: {X_sliced.shape}")

## Transform Pipeline

In [ ]:
class TSAugmenter:    """Time series augmentation pipeline."""        def __init__(self, noise_scale=0.1, mag_scale_range=0.2, p=0.5):        self.noise_scale = noise_scale        self.mag_scale_range = mag_scale_range        self.p = p  # Probability of applying each transform        def __call__(self, X):        """Apply random augmentations."""        X = np.asarray(X).copy()                # Apply Gaussian noise with probability p        if np.random.random() < self.p:            X = tsai_rs.add_gaussian_noise(X, scale=self.noise_scale)                # Apply magnitude scaling with probability p        if np.random.random() < self.p:            X = tsai_rs.mag_scale(X, scale=self.mag_scale_range)                return X

In [ ]:
# Test augmenteraugmenter = TSAugmenter(noise_scale=0.1, mag_scale_range=0.2, p=0.5)X_aug = augmenter(X_train_std[:10])print(f"Augmented shape: {X_aug.shape}")print(f"Original mean: {np.mean(X_train_std[:10]):.4f}")print(f"Augmented mean: {np.mean(X_aug):.4f}")